# Kubeflow Fairing Introduction

Kubeflow Fairing is a Python package that streamlines the process of `building`, `training`, and `deploying` machine learning (ML) models in a hybrid cloud environment. By using Kubeflow Fairing and adding a few lines of code, you can run your ML training job locally or in the cloud, directly from Python code or a Jupyter notebook. After your training job is complete, you can use Kubeflow Fairing to deploy your trained model as a prediction endpoint.



# How does Kubeflow Fairing work

Kubeflow Fairing 
1. Packages your Jupyter notebook, Python function, or Python file as a Docker image
2. Deploys and runs the training job on Kubeflow or AI Platform. 
3. Deploy your trained model as a prediction endpoint on Kubeflow after your training job is complete.


# Goals of Kubeflow Fairing project

- Easily package ML training jobs: Enable ML practitioners to easily package their ML model training code, and their code’s dependencies, as a Docker image.
- Easily train ML models in a hybrid cloud environment: Provide a high-level API for training ML models to make it easy to run training jobs in the cloud, without needing to understand the underlying infrastructure.
- Streamline the process of deploying a trained model: Make it easy for ML practitioners to deploy trained ML models to a hybrid cloud environment.



In [ ]:
# Option 1: Install Fairing Python SDK from python repository
!pip install fairing

In [ ]:
# Option 2: Install latest Fairing from github repository (If you'd like to try new features)
!git clone https://github.com/kubeflow/fairing 
!pip install --upgrade fairing/

In [ ]:
# check fairing is installed 
!pip show fairing

## Basic Example

In [8]:
import os
import sys
import fairing
import tensorflow as tf

def train():
    if 'HOSTNAME' in os.environ:
        hostname = tf.constant(os.environ['HOSTNAME'])
    else:
        hostname = "fake_host_name"
    sess = tf.Session()
    print('Hostname: ', sess.run(hostname).decode('utf-8'))

In [9]:
# Local training for development
train()

Hostname:  test-rbac-0


In [ ]:
# Remote Training
remote_train = fairing.config.fn(train)
remote_train()

## DockerHub Example

In [ ]:
# Make sure authenticate your DockerHub Registry

In [ ]:
import os
import fairing
import tensorflow as tf

# Setting up DockerHub container repositories for storing output containers
DOCKERHUB_USERNAME = 'seedjeffwan'
DOCKER_REGISTRY = '{}'.format(DOCKERHUB_USERNAME)
fairing.config.set_builder('append', base_image='tensorflow/tensorflow:1.14.0-py3', registry=DOCKER_REGISTRY, push=True)
fairing.config.set_deployer('job')

def train():
    if 'HOSTNAME' in os.environ:
        hostname = tf.constant(os.environ['HOSTNAME'])
    else:
        hostname = "fake_host_name"
    sess = tf.Session()
    print('Hostname: ', sess.run(hostname).decode('utf-8'))

if __name__ == '__main__':
    remote_train = fairing.config.fn(train)
    remote_train()

## ECR Example

In [ ]:
# Authenticate ECR
# Please copy output of this command and run. 

!aws ecr get-login --no-include-email

In [ ]:
# Don't forget to give a `!` before your command to make interpreter understand it's a shell command


# After you run the command, you will see 
# -------
# WARNING! Using --password via the CLI is insecure. Use --password-stdin.
# Login Succeeded
# -------

In [ ]:
import os
import sys
import fairing
import tensorflow as tf


# Setting up AWS Elastic Container Registry (ECR) for storing output containers
# You can use any docker container registry istead of ECR
AWS_ACCOUNT_ID=fairing.cloud.aws.guess_account_id()
AWS_REGION='us-west-2'
DOCKER_REGISTRY = '{}.dkr.ecr.{}.amazonaws.com'.format(AWS_ACCOUNT_ID, AWS_REGION)
PY_VERSION = ".".join([str(x) for x in sys.version_info[0:3]])
BASE_IMAGE = 'python:{}'.format(PY_VERSION)

fairing.config.set_builder('append', base_image='tensorflow/tensorflow:1.14.0-py3', registry=DOCKER_REGISTRY, push=True)
fairing.config.set_deployer('job')

def train():
    if 'HOSTNAME' in os.environ:
        hostname = tf.constant(os.environ['HOSTNAME'])
    else:
        hostname = "fake_host_name"
    sess = tf.Session()
    print('Hostname: ', sess.run(hostname).decode('utf-8'))
    
if __name__ == '__main__':
    remote_train = fairing.config.fn(train)
    remote_train()